## Лабораторная работа 4
by Константин Киселев, МОА-211

In [63]:
import secrets
import hashlib
import os

### Задание 1
Возьмите реализацию класса HashTable из лекционных материалов и выполните
следующие доработки:
1. Реализуйте квадратичное пробирование как технику повторного хеширования.
2. Реализуйте работу с функцией len (переопределите метод __len__).
3. Реализуйте работу оператора in (переопределите метод __contains__).
4. Переделайте метод put таким образом, чтобы таблица автоматически меняла размер,
когда загрузочный фактор становится больше значения 0.7. Размер должен
увеличиваться примерно в два раза до ближайшего подходящего простого числа.
5. Реализуйте работу оператора del (переопределите метод __delitem__) для удаления
элемента таблицы. Таблица должна автоматически менять размер, когда
загрузочный фактор становится меньше значения 0.2. Размер должен уменьшаться
примерно в два раза до ближайшего подходящего простого числа.

Все выполненные доработки должны быть протестированы.

In [64]:
class HashTable:
    def __init__(self):
        self.size = 11
        self.MAX_LOAD_FACTOR = 0.7
        self.MIN_LOAD_FACTOR = 0.2
        self.slots = [None] * self.size
        self.data = [None] * self.size

    def _is_prime(self, n):
        if n <= 1:
            return False
        if n <= 3:
            return True
        if n % 2 == 0 or n % 3 == 0:
            return False
        i = 5
        while i * i <= n:
            if n % i == 0 or n % (i + 2) == 0:
                return False
            i += 6
        return True

    def put(self, key, data):
        self.load_factor = len(self) / len(self.slots)
        if self.load_factor > self.MAX_LOAD_FACTOR:
            self.resize()

        hashvalue = self.hashfunction(key, len(self.slots))

        if self.slots[hashvalue] == None:
            self.slots[hashvalue] = key
            self.data[hashvalue] = data
        else:
            if self.slots[hashvalue] == key:
                self.data[hashvalue] = data  # replace
            else:
                count = 1
                nextslot = self.rehash(hashvalue, len(self.slots), 1)
                while self.slots[nextslot] != None and self.slots[nextslot] != key:
                    count += 1
                    nextslot = self.rehash(nextslot, len(self.slots), count)

                if self.slots[nextslot] == None:
                    self.slots[nextslot] = key
                    self.data[nextslot] = data
                else:
                    self.data[nextslot] = data  # replace

    def hashfunction(self, key, size):
        return key % size

    def rehash(self, oldhash, size, count):
        return (oldhash + count**2) % size

    def get(self, key):
        startslot = self.hashfunction(key, len(self.slots))

        data = None
        stop = False
        found = False
        position = startslot
        count = 1
        while self.slots[position] != None and not found and not stop:
            if self.slots[position] == key:
                found = True
                data = self.data[position]
            else:
                position = self.rehash(position, len(self.slots), count)
                count += 1
                if position == startslot:
                    stop = True
        return data

    def resize(self, backward=False, rehash=False):
        new_size = self.size
        if not rehash:
            new_size = self.size // 2 if backward else self.size * 2
            step = -1 if backward else 1
            while not self._is_prime(new_size):
                new_size += step

        slots = list(filter(None, self.slots))
        data = list(filter(None, self.data))

        self.slots = [None] * new_size
        self.data = [None] * new_size
        self.size = new_size

        for key, data in dict(zip(slots, data)).items():
            self.put(key, data)

    def __getitem__(self, key):
        return self.get(key)

    def __setitem__(self, key, data):
        self.put(key, data)

    def __len__(self):
        return len(list(filter(None, self.slots)))

    def __contains__(self, key):
        return self.get(key) is not None

    def __delitem__(self, key):
        self.load_factor = len(self) / len(self.slots)

        if self.load_factor < self.MIN_LOAD_FACTOR:
            self.resize(backward=True)

        startslot = self.hashfunction(key, len(self.slots))
        count = 1
        found = False
        position = startslot
        while not found:
            if self.slots[position] == key:
                self.slots[position] = None
                self.data[position] = None
                found = True
            else:
                position = self.rehash(startslot, len(self.slots), count)
                count += 1
                if position == startslot:
                    break

        self.resize(rehash=True)


h = HashTable()
print(f"start size: {len(h)}")
h[44] = "dog"
h[55] = "cat"
h[66] = "fish"
h[98] = "a"
h[99] = "b"
h[100] = "c"
h[101] = "d"
h[102] = "e"
print(f"size after adding elements: {len(h)}\n")
print(f"current view of table:\n{h.slots}\n{h.data}\n")
print('"in" test: 66 in h: {66 in h}\n')
h[103] = "f"
print(f"forward resize test:\n{h.slots}\n{h.data}\n")
del h[103]
del h[55]
del h[101]
del h[102]
del h[100]
del h[99]
print(f"backward resize test:\n{h.slots}\n{h.data}\n")
print(f"get test after resize: {h[66]}")

start size: 0
size after adding elements: 8

current view of table:
[44, 55, 100, 99, 102, 66, None, 101, None, None, 98]
['dog', 'cat', 'c', 'b', 'e', 'fish', None, 'd', None, None, 'a']

"in" test: 66 in h: {66 in h}

forward resize test:
[None, None, None, None, None, None, 98, 99, 100, 55, 102, 103, None, None, 101, None, None, None, None, None, 66, 44, None]
[None, None, None, None, None, None, 'a', 'b', 'c', 'cat', 'e', 'f', None, None, 'd', None, None, None, None, None, 'fish', 'dog', None]

backward resize test:
[66, 44, None, None, None, None, None, None, None, None, 98]
['fish', 'dog', None, None, None, None, None, None, None, None, 'a']

get test after resize: fish


### Задание 2
Возьмите реализацию класса HashTable из лекционных материалов и выполните
следующие доработки:
1. Переделайте существующие методы так, чтобы разрешение коллизий происходило
не при помощи концепции открытой адресации, а методом цепочек. Для этого в
каждом слоте храните связный список, реализованный классом UnorderedList из
лабораторной работы 3.
2. Реализуйте работу с функцией len (переопределите метод __len__).
3. Реализуйте работу оператора in (переопределите метод __contains__).
4. Переделайте метод put таким образом, чтобы таблица автоматически меняла размер,
когда загрузочный фактор становится больше значения 0.7. Размер должен
увеличиваться примерно в два раза до ближайшего подходящего простого числа.
5. Реализуйте работу оператора del (переопределите метод __delitem__) для удаления
элемента таблицы. Таблица должна автоматически менять размер, когда
загрузочный фактор становится меньше значения 0.2. Размер должен уменьшаться
примерно в два раза до ближайшего подходящего простого числа.
Все выполненные доработки должны быть протестированы.

In [65]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

    def getData(self):
        return self.data

    def getNext(self):
        return self.next

    def setData(self, data):
        self.data = data

    def setNext(self, next):
        self.next = next


class UnorderedList:
    def __init__(self, item=None):
        self.head = Node(item)

    def isEmpty(self):
        return self.head is None

    def add(self, item):
        temp = Node(item)
        temp.setNext(self.head)
        self.head = temp

    def size(self):
        current = self.head
        count = 0
        while current is not None:
            count = count + 1
            current = current.getNext()

        return count

    def search(self, item):
        current = self.head
        found = False
        while current is not None and not found:
            if current.getData() == item:
                found = True
            else:
                current = current.getNext()

        return found

    def remove(self, item):
        current = self.head
        previous = None
        found = False
        while not found:
            if current.getData() == item:
                found = True
            else:
                previous = current
                current = current.getNext()

        if previous is None:
            self.head = current.getNext()
        else:
            previous.setNext(current.getNext())

    def append(self, item):
        new_node = Node(item)
        if self.head is None:
            self.head = new_node
        else:
            current = self.head
            while current.getNext() is not None:
                current = current.getNext()
            current.setNext(new_node)

    def insert(self, pos, item):
        if pos == 0:
            self.add(item)
        else:
            current = self.head
            previous = None
            idx = 0
            while idx < pos and current is not None:
                previous = current
                current = current.getNext()
                idx += 1

            new_node = Node(item)
            new_node.setNext(current)
            previous.setNext(new_node)

    def index(self, item):
        current = self.head
        idx = -1

        while current is not None:
            idx += 1
            if current.getData() == item:
                break
            else:
                current = current.getNext()
        else:
            idx = -1

        return idx

    def pop(self, pos=None):
        if pos is None:
            current = self.head

            if current is None:
                return current
            elif current.getNext() is None:
                self.head = None
                return current.getData()

            while current.getNext().getNext() is not None:
                current = current.getNext()

            temp = current.getNext()
            current.setNext(None)

            return temp.getData()
        else:
            if pos < 0 or pos >= self.size():
                raise ValueError("Invalid position")

            current = self.head
            previous = None
            index = 0

            while index < pos:
                previous = current
                current = current.getNext()
                index += 1

            if previous:
                previous.setNext(current.getNext())
            else:
                self.head = current.getNext()

            return current.getData()

    def __str__(self):
        current = self.head
        result = "["

        while current is not None:
            result += str(current.getData())
            if current.getNext() is not None:
                result += ", "
            current = current.getNext()

        return result + "]"

    def slice(self, start, stop):
        result = UnorderedList()
        current = self.head
        index = 0

        while index < start and current is not None:
            current = current.getNext()
            index += 1

        while index < stop and current is not None:
            result.append(current.getData())
            current = current.getNext()
            index += 1

        return result

    def list(self):
        current = self.head
        result = []

        while current is not None:
            result.append(current)
            current = current.getNext()

        return result


class HashTable:
    def __init__(self):
        self.size = 11
        self.MAX_LOAD_FACTOR = 0.7
        self.MIN_LOAD_FACTOR = 0.2
        self.slots = [None] * self.size
        self.data = [None] * self.size

    def _is_prime(self, n):
        if n <= 1:
            return False
        if n <= 3:
            return True
        if n % 2 == 0 or n % 3 == 0:
            return False
        i = 5
        while i * i <= n:
            if n % i == 0 or n % (i + 2) == 0:
                return False
            i += 6
        return True

    def put(self, key, data):
        self.load_factor = len(self) / len(self.slots)

        if self.load_factor > self.MAX_LOAD_FACTOR:
            self.resize()

        hashvalue = self.hashfunction(key, len(self.slots))

        if self.slots[hashvalue] is None:
            self.slots[hashvalue] = UnorderedList(key)
            self.data[hashvalue] = UnorderedList(data)
        else:
            idx = self.slots[hashvalue].index(key)
            if idx != -1:
                self.data[hashvalue].pop(idx)
                self.data[hashvalue].insert(idx, data)
            else:
                self.slots[hashvalue].append(key)
                self.data[hashvalue].append(data)

    def get(self, key):
        position = self.hashfunction(key, len(self.slots))
        data = None
        if self.slots[position] is not None:
            idx = self.slots[position].index(key)
            if idx != -1:
                data = self.data[position].pop(idx)
                self.data[position].insert(idx, data)
        return data

    def hashfunction(self, key, size):
        return key % size

    def rehash(self, slots, data):
        for slot_list, data_list in dict(zip(slots, data)).items():
            for key, data in dict(zip(slot_list.list(), data_list.list())).items():
                self.put(key.getData(), data.getData())

    def resize(self, backward=False):
        new_size = self.size // 2 if backward else self.size * 2
        step = -1 if backward else 1
        while not self._is_prime(new_size):
            new_size += step

        slots = list(filter(None, self.slots))
        data = list(filter(None, self.data))

        self.slots = [None] * new_size
        self.data = [None] * new_size
        self.size = new_size

        self.rehash(slots, data)

    def __getitem__(self, key):
        return self.get(key)

    def __setitem__(self, key, data):
        self.put(key, data)

    def __len__(self):
        size = 0
        for lst in self.slots:
            if lst is not None:
                size += lst.size()
        return size

    def __contains__(self, key):
        return self.get(key) is not None

    def __delitem__(self, key):
        self.load_factor = len(self) / len(self.slots)

        if self.load_factor < self.MIN_LOAD_FACTOR:
            self.resize(backward=True)

        position = self.hashfunction(key, len(self.slots))
        if self.slots[position] is not None:
            idx = self.slots[position].index(key)
            if idx != -1:
                self.slots[position].pop(idx)
                self.data[position].pop(idx)

        if self.slots[position].isEmpty():
            self.slots[position] = None
            self.data[position] = None


def print_table(h):
    for slot in h.slots:
        print(slot, end=" ")
    print()
    for data in h.data:
        print(data, end=" ")
    print("\n")


h = HashTable()
print(f"start size: {len(h)}")
h[44] = "dog"
h[55] = "cat"
h[66] = "fish"
h[98] = "a"
h[99] = "b"
h[100] = "c"
h[101] = "d"
h[102] = "e"
print(f"size after adding elements: {len(h)}\n")
print("current view of table:")
print_table(h)
print('"in" test: 66 in h: {66 in h}\n')
h[103] = "f"
print("forward resize test:")
print_table(h)
del h[103]
del h[55]
del h[101]
del h[102]
del h[100]
del h[99]
print(f"backward resize test:")
print_table(h)
print(f"get test after resize: {h[66]}")

start size: 0
size after adding elements: 8

current view of table:
[44, 55, 66, 99] [100] [101] [102] None None None None None None [98] 
[dog, cat, fish, b] [c] [d] [e] None None None None None None [a] 

"in" test: 66 in h: {66 in h}

forward resize test:
None None None None None None [98] [99] [100] [55, 101] [102] [103] None None None None None None None None [66] [44] None 
None None None None None None [a] [b] [c] [cat, d] [e] [f] None None None None None None None None [fish] [dog] None 

backward resize test:
[66, 44] None None None None None None None None None [98] 
[fish, dog] None None None None None None None None None [a] 

get test after resize: fish


### Задание 3
Переделайте класс HashTable, чтобы в качестве ключей можно было использовать строки.

In [66]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

    def getData(self):
        return self.data

    def getNext(self):
        return self.next

    def setData(self, data):
        self.data = data

    def setNext(self, next):
        self.next = next


class UnorderedList:
    def __init__(self, item=None):
        self.head = Node(item)

    def isEmpty(self):
        return self.head is None

    def add(self, item):
        temp = Node(item)
        temp.setNext(self.head)
        self.head = temp

    def size(self):
        current = self.head
        count = 0
        while current is not None:
            count = count + 1
            current = current.getNext()

        return count

    def search(self, item):
        current = self.head
        found = False
        while current is not None and not found:
            if current.getData() == item:
                found = True
            else:
                current = current.getNext()

        return found

    def remove(self, item):
        current = self.head
        previous = None
        found = False
        while not found:
            if current.getData() == item:
                found = True
            else:
                previous = current
                current = current.getNext()

        if previous is None:
            self.head = current.getNext()
        else:
            previous.setNext(current.getNext())

    def append(self, item):
        new_node = Node(item)
        if self.head is None:
            self.head = new_node
        else:
            current = self.head
            while current.getNext() is not None:
                current = current.getNext()
            current.setNext(new_node)

    def insert(self, pos, item):
        if pos == 0:
            self.add(item)
        else:
            current = self.head
            previous = None
            idx = 0
            while idx < pos and current is not None:
                previous = current
                current = current.getNext()
                idx += 1

            new_node = Node(item)
            new_node.setNext(current)
            previous.setNext(new_node)

    def index(self, item):
        current = self.head
        idx = -1

        while current is not None:
            idx += 1
            if current.getData() == item:
                break
            else:
                current = current.getNext()
        else:
            idx = -1

        return idx

    def pop(self, pos=None):
        if pos is None:
            current = self.head

            if current is None:
                return current
            elif current.getNext() is None:
                self.head = None
                return current.getData()

            while current.getNext().getNext() is not None:
                current = current.getNext()

            temp = current.getNext()
            current.setNext(None)

            return temp.getData()
        else:
            if pos < 0 or pos >= self.size():
                raise ValueError("Invalid position")

            current = self.head
            previous = None
            index = 0

            while index < pos:
                previous = current
                current = current.getNext()
                index += 1

            if previous:
                previous.setNext(current.getNext())
            else:
                self.head = current.getNext()

            return current.getData()

    def __str__(self):
        current = self.head
        result = "["

        while current is not None:
            result += str(current.getData())
            if current.getNext() is not None:
                result += ", "
            current = current.getNext()

        return result + "]"

    def slice(self, start, stop):
        result = UnorderedList()
        current = self.head
        index = 0

        while index < start and current is not None:
            current = current.getNext()
            index += 1

        while index < stop and current is not None:
            result.append(current.getData())
            current = current.getNext()
            index += 1

        return result

    def list(self):
        current = self.head
        result = []

        while current is not None:
            result.append(current)
            current = current.getNext()

        return result


class HashTable:
    def __init__(self):
        self.size = 11
        self.MAX_LOAD_FACTOR = 0.7
        self.MIN_LOAD_FACTOR = 0.2
        self.slots = [None] * self.size
        self.data = [None] * self.size

    def _is_prime(self, n):
        if n <= 1:
            return False
        if n <= 3:
            return True
        if n % 2 == 0 or n % 3 == 0:
            return False
        i = 5
        while i * i <= n:
            if n % i == 0 or n % (i + 2) == 0:
                return False
            i += 6
        return True

    def put(self, key, data):
        self.load_factor = len(self) / len(self.slots)

        if self.load_factor > self.MAX_LOAD_FACTOR:
            self.resize()

        hashvalue = self.hashfunction(key, len(self.slots))

        if self.slots[hashvalue] is None:
            self.slots[hashvalue] = UnorderedList(key)
            self.data[hashvalue] = UnorderedList(data)
        else:
            idx = self.slots[hashvalue].index(key)
            if idx != -1:
                self.data[hashvalue].pop(idx)
                self.data[hashvalue].insert(idx, data)
            else:
                self.slots[hashvalue].append(key)
                self.data[hashvalue].append(data)

    def get(self, key):
        position = self.hashfunction(key, len(self.slots))
        data = None
        if self.slots[position] is not None:
            idx = self.slots[position].index(key)
            if idx != -1:
                data = self.data[position].pop(idx)
                self.data[position].insert(idx, data)
        return data

    def hashfunction(self, key, size):
        res = 0
        if isinstance(key, str):
            idx = 0
            for symbol in key:
                res += ord(symbol) * idx
                idx += 1
        if isinstance(key, int):
            res = key
        return res % size

    def rehash(self, slots, data):
        for slot_list, data_list in dict(zip(slots, data)).items():
            for key, data in dict(zip(slot_list.list(), data_list.list())).items():
                self.put(key.getData(), data.getData())

    def resize(self, backward=False):
        new_size = self.size // 2 if backward else self.size * 2
        step = -1 if backward else 1
        while not self._is_prime(new_size):
            new_size += step

        slots = list(filter(None, self.slots))
        data = list(filter(None, self.data))

        self.slots = [None] * new_size
        self.data = [None] * new_size
        self.size = new_size

        self.rehash(slots, data)

    def __getitem__(self, key):
        return self.get(key)

    def __setitem__(self, key, data):
        self.put(key, data)

    def __len__(self):
        size = 0
        for lst in self.slots:
            if lst is not None:
                size += lst.size()
        return size

    def __contains__(self, key):
        return self.get(key) is not None

    def __delitem__(self, key):
        self.load_factor = len(self) / len(self.slots)

        if self.load_factor < self.MIN_LOAD_FACTOR:
            self.resize(backward=True)

        position = self.hashfunction(key, len(self.slots))
        if self.slots[position] is not None:
            idx = self.slots[position].index(key)
            if idx != -1:
                self.slots[position].pop(idx)
                self.data[position].pop(idx)

        if self.slots[position].isEmpty():
            self.slots[position] = None
            self.data[position] = None


def print_table(h):
    for slot in h.slots:
        print(slot, end=" ")
    print()
    for data in h.data:
        print(data, end=" ")
    print("\n")


h = HashTable()
print("start size:", len(h))
h[44] = "dog"
h[55] = "cat"
h[66] = "fish"
h["test1"] = "a"
h["test2"] = "b"
h["test3"] = "c"
h["test4"] = "d"
h["test5"] = "e"
print(f"size after adding elements:{len(h)}\n")
print("current view of table:")
print_table(h)
print(f'"in" test: 66 in h: {66 in h}\n')
h[103] = "f"
print("forward resize test:")
print_table(h)
del h[103]
del h[55]
del h["test4"]
del h["test5"]
del h["test3"]
del h["test2"]
print("backward resize test:")
print_table(h)
print("get test after resize:", h[66])

start size: 0
size after adding elements:8

current view of table:
[44, 55, 66, test5] None None [test3] None None [test1] [test4] None None [test2] 
[dog, cat, fish, e] None None [c] None None [a] [d] None None [b] 

"in" test: 66 in h: True

forward resize test:
None [test1] None None None [test2] None None None [55, test3] None [103] None [test4] None None None [test5] None None [66] [44] None 
None [a] None None None [b] None None None [cat, c] None [f] None [d] None None None [e] None None [fish] [dog] None 

backward resize test:
[66, 44] None None None None None [test1] None None None None 
[fish, dog] None None None None None [a] None None None None 

get test after resize: fish


### Задание 4
Дана строчка русского текста, состоящая из слов и пробелов. Словом считается
последовательность русских букв, слова разделены одним или большим числом пробелов.

Для каждого слова этого текста узнайте порядковый номер его вхождения в текст именно в
той форме, в которой указано слово. Для первого вхождения слова выведите «1», для
второго вхождения того же слова выведите «2» и так далее.

Для решения этой задачи используйте класс HashTable из задания № 3.

Пример

Ввод

Раз раз раз как меня слышно Повторяю раз раз раз Повторяю

Вывод

1 1 2 1 1 1 1 3 4 5 2

In [67]:
s = input("введите строку: ")
h = HashTable()
for i in s.split():
    if i in h:
        h[i] += 1
    else:
        h[i] = 1
    print(h[i], end=" ")

1 1 2 1 1 1 1 3 4 5 2 

### Задание 5
Напишите программу, имитирующую процесс регистрации и авторизации. Для каждого
пользователя программа должна сохранять логин, хеш его пароля и «соль». Для хранения
данных можно использовать БД или файл.
Действия при сохранении пароля:
1. Сгенерируйте длинную случайную «соль» при помощи модуля secrets (secrets —
Generate secure random numbers for managing secrets — Python 3.10.0 documentation).
Длина «соли» должна быть такой же как и выходные данные используемой вами
хэш-функции. Например, если для хеширования вы используете SHA256, то на
выходе вы получите 256 бит (32 байта). В этом случае соль должна составлять не
менее 32 случайных байт.
2. Добавьте «соль» к паролю и хэшируйте его с помощью функции scrypt из модуля
hashlib (Хеширование паролей модулем hashlib в Python. (docs-python.ru)).
3. Сохраните логин, «соль» и получившейся хэш в БД или в файл.
Действия при проверке пароля:
1. Извлеките «соль» и хэш пользователя из БД или файла.
2. Добавьте «соль» к введенному паролю и хэшируйте его, используя ту же хэшфункцию, что и в алгоритме сохранения пароля.
3. Сравните получившейся хэш введенного пароля с хэшом из БД или файла. Если они
совпадают, то пароль правильный. В противном случае пароль был введен неверно. 

In [68]:
def register():
    login = input("введите логин: ")
    password = input("введите пароль: ")

    salt = secrets.token_hex(32)
    hashed_password = hashlib.scrypt(
        password.encode(), salt=salt.encode(), n=8, r=512, p=4, dklen=32
    ).hex()

    with open(f"./files/{login}.txt", "w") as file:
        file.write(f"{salt}:{hashed_password}\n")

    print("регистрация выполнена успешно.")


def login():
    login = input("введите логин: ")
    password = input("введите пароль: ")

    try:
        with open(f"./files/{login}.txt", "r") as file:
            file_salt, file_password = file.readline().strip().split(":")

            hashed_password = hashlib.scrypt(
                password.encode(), salt=file_salt.encode(), n=8, r=512, p=4, dklen=32
            ).hex()

            if hashed_password == file_password:
                print("авторизация выполнена успешно.")
            else:
                print("неверный пароль.")
    except FileNotFoundError:
        print("такой пользователь не существует.")


register()
login()

регистрация выполнена успешно.
авторизация выполнена успешно.


### Задание 6
Напишите программу, которая принимает от пользователя путь до директории. Для всех
файлов из данной директории должен быть вычислен хеш. Программа должна выявить и
вывести на экран все дубликаты в этой директории (т.е. файлы, у которых одинаковый хеш).

In [72]:
def get_file_hash(file_path):
    with open(file_path, "rb") as file:
        return hashlib.sha256(file.read()).hexdigest()


def find_duplicates(directory):
    dct = {}

    for root, dirs, files in os.walk(directory):
        for file in files:
            path = os.path.join(root, file)
            hash = get_file_hash(path)
            if dct.get(hash) is None:
                dct[hash] = []
            dct[hash].append(path)

    for file_hash, file_paths in dct.items():
        if len(file_paths) > 1:
            print(f"дубликаты для хеша {file_hash}:")
            for file_path in file_paths:
                print(file_path)
            print()


def main():
    directory = input("введите путь до директории: ")
    find_duplicates(directory)


main()

дубликаты для хеша f77d701e347d2304521c318444eb20c172a8223d2ff6cb3feafbd959718c95de:
./files/admin.txt
./files/user.txt

